In [2]:
from src.reader import Reader
from src.downloader import Downloader

In [ ]:
#Download images Starting from 1st challange to 10th challange
#In other words dowload 10 subsets
downloader = Downloader(start=11, end=100)
downloader.download(path="data/AVA.txt")

In [4]:
downloader = Downloader(start=201, end=202)
downloader.download(path="data/AVA.txt")